<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Orbiting" data-toc-modified-id="Orbiting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Orbiting</a></span></li><li><span><a href="#Solar-spectrum" data-toc-modified-id="Solar-spectrum-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Solar spectrum</a></span></li></ul></div>

In [1]:
%matplotlib ipympl

# Orbiting

Combining slew rate with surface travel rates etc.

In [2]:
from astropy import units as u
from pytelescope import orbiter
orb = orbiter.MarsOrbiter(350)

Assuming kilometers as unit for input parameter.


In [3]:
orb.v

<Quantity 3384.2089663 m / s>

# Reflectances

In [4]:
import astropy.units as u
from astropy.constants import h, c
from scipy.interpolate import InterpolatedUnivariateSpline
import math

In [5]:
rootpath = Path("/Users/klay6683/Documents/proposals/2018/MAPSE/")

In [6]:
def interpolate(rsr, target_waves):
    ius = InterpolatedUnivariateSpline(rsr['wavelength'], 
                                       rsr['response'],
                                       k=1)
    return ius(target_waves)


class Albedo:
    def __init__(self, csvfilepath):
        df = pd.read_csv(csvfilepath)
        self.albedo = df.sort_values(by='Wavelength[nm]')

    @property
    def rsr(self):
        d = {}
        d['wavelength'] = self.albedo.iloc[:, 0]
        d['response'] = self.albedo.iloc[:, 1]
        return d
    
    def resp_ipol(self, target_waves):
        return interpolate(self.rsr, target_waves )

    @property
    def wave1(self):
        return self.albedo.iloc[0, 0] * u.nm
    
    @property
    def wave2(self):
        return self.albedo.iloc[-1, 0] * u.nm

# Requirements
1.       SNR 100 is a good planning value for the camera.
2.       Filter wavelengths. Dayside: 200, 285, 365, 550, 935 (TBR); Nightside: 1.02, 1.72, 2.26, 2.32
3.       Filter widths are not so critical, use VMC or Akatsuki as baseline.
4.       Spatial resolution of 10 km at the cloud tops is OK.
5.       Observing scenario. Dayside global imaging and spectroscopy: Every 30 minutes for 2 hours, wait 4 hours, repeat this pattern twice; Evening, morning, nightside: every 4 hours. High cadence campaign: Short bursts of images every 5 minutes on dayside, for 1 h. Downlink once/orbit on nightside of Venus, 8 hours.
6.       Retrograde orbit is better (this is the same direction Venus rotates and the clouds super-rotate). Don’t aim for ‘cloud-top synchronization,’ the cloud speed varies with time, altitude and latitude. Orbital period 3-5 days is acceptable. 40 degree orbit inclination is OK, gives coverage of equatorial and mid-latitudes, with some view of the polar regions. LWE likes 3day period.
7.       Orbit eccentricity < 0.5.
8.       The camera capabilities can look for hot spots on the surface as volcanic evidence, as VMC reported. No additional requirements. The spectrometers will look for sudden increases in H2O and SO2 etc.


# Solar spectrum

In [7]:
from pyspectral.solar import (SolarIrradianceSpectrum, 
                              TOTAL_IRRADIANCE_SPECTRUM_2000ASTM,
                             )

In [8]:
dic = {
    'A_t': 5e-3 *u.m*u.m,
    'A_p': 1.69e-10 *u.m*u.m,
    'f' : 1.3 * u.m,
    'T_M1':0.92,
    'T_M2':0.92,
    'T_s': 0.94,
}


class Radiometry:
    E_w_unit_in = u.Watt/u.m/u.m/u.micron
    E_w_unit_out = u.Watt/u.m/u.m/u.nm
    E_ph_unit = 1/(u.s*u.m*u.m*u.nm)
    plot_ctrl = dict(lw=0.75)
    rootpath = Path("/Users/klay6683/Documents/proposals/2018/MAPSE/")
    def __init__(self, albedo, wave1=200*u.nm, wave2=1200*u.nm, dlambda=1*u.nm,
                 i=75*u.deg, d=1.5):
        self.albedo = albedo
        self.wave1 = wave1
        self.wave2 = wave2
        self.dlambda = dlambda
        self.i = i  # incidence angle
        self.d = d  # Mars distance in AU (scaling the solar flux)
        sol = SolarIrradianceSpectrum(TOTAL_IRRADIANCE_SPECTRUM_2000ASTM)
        sol.interpolate(dlambda=dlambda.to(u.micron).value, 
                        ival_wavelength=(wave1.to(u.micron).value,
                                         wave2.to(u.micron).value))
        self.waves = (sol.ipol_wavelength*u.micron).to(u.nm)
        self.E_w = (sol.ipol_irradiance*self.E_w_unit_in).to(self.E_w_unit_out)
        
        self.read_QE()
        for k,v in dic.items():
            setattr(self, k, v)
                
    def read_QE(self):
        df = pd.read_csv(self.rootpath / 'midband_coated_QE.csv')
        df = df.sort_values(by='Wavelength[nm]')
        pre_data = pd.DataFrame([[225, 0.0]], columns=df.columns)
        post_data = pd.DataFrame([[1100, 0.0]], columns=df.columns)
        self.QE = pd.concat([pre_data, df, post_data])
        
    @property
    def QE_rsr(self):
        d = {}
        d['wavelength'] = self.QE.iloc[:, 0]
        d['response'] = self.QE.iloc[:, 1]/100.0
        return d 
    
    @property
    def rsr(self):
        return self.albedo.rsr
    
    def plot_E_w(self, ax=None):
        xlim = [self.wave1.value, self.wave2.value]
        
        if ax is None:
            _, ax = plt.subplots(figsize=(8,4))

        ax.plot(self.waves, self.E_w, **self.plot_ctrl)
        ax.set_xlim(xlim)
        ax.set_ylim(0, 2.5)
        ax.grid(True)
        ax.set_xlabel(f"Wavelength [{self.waves.unit}]")
        ax.set_ylabel(f"Spectral irradiance [{self.E_w.unit}]")
        ax.set_title("E490 Spectral irradiance ($E_w$)")
        
    @property
    def ph_per_energy(self):
        return self.waves/(h*c)
    
    @property
    def E_ph(self):
        return (self.E_w * self.ph_per_energy).to(self.E_ph_unit)
    
    def plot_E_ph(self, ax=None):
        xlim = [self.wave1.value, self.wave2.value]
        
        if ax is None:
            _, ax = plt.subplots(figsize=(8,4))

        ax.plot(self.waves, self.E_ph, **self.plot_ctrl)
        ax.set_xlim(xlim)
        ax.set_ylim(ymin=0, ymax=6e18)
        ax.grid(True)
        ax.set_xlabel(f"Wavelength [{self.waves.unit}]")
        ax.set_ylabel(f"Spectral irradiance [{self.E_ph.unit}]")
        ax.set_title("E490 Spectral irradiance ($E_{ph}$)")
              
    @property
    def resp_ipol(self):
        return self.albedo.resp_ipol(self.waves.value)
    
    @property
    def QE_ipol(self):
        ius = InterpolatedUnivariateSpline(self.QE_rsr['wavelength'],
                                           self.QE_rsr['response'],
                                           k=1)
        return ius(self.waves.value)
        
    @property
    def L_surf(self):
        term1 = self.E_ph/self.d**2
        term2 = math.cos(self.i.to(u.rad).value) / math.pi
        term3 = self.resp_ipol
        return term1*term2*term3
    
    @property
    def CR(self):
        term1 = self.L_surf * self.A_t * self.A_p
        term2 = self.T_M1 * self.T_M2 * self.T_s * self.QE_ipol
        return term1*term2/(self.f**2)
    
    @property
    def signal_rate(self):
        return self.CR.sum()
    
    def SNR(self, exp=0.01):
        return math.sqrt(self.signal_rate.value * exp)

In [9]:
alb = Albedo("/Users/klay6683/Documents/VENUS/VDO/low_mid_lat_albedo.csv")

In [10]:
radio = Radiometry(albedo=alb, wave1=alb.wave1, wave2=alb.wave2,
                   d=0.723)

In [11]:
radio.SNR(0.0015)

261.2363183862308

In [12]:
plt.figure()
plt.plot(radio.waves, radio.CR)
plt.grid()

FigureCanvasNbAgg()

In [13]:
fig, ax = plt.subplots(ncols=2, figsize=(8,3))
radio.plot_E_w(ax[0])
radio.plot_E_ph(ax[1])

FigureCanvasNbAgg()

In [14]:
plt.close('all')

In [15]:
plt.figure()
plt.plot(radio.rsr['wavelength'],
         radio.rsr['response'])

FigureCanvasNbAgg()

In [16]:
d = 80 * u.mm

In [17]:
A = math.pi * d**2/4

In [18]:
A.to(u.m*u.m)

<Quantity 0.00502655 m2>

In [19]:
plt.figure()
plt.plot(radio.QE_rsr['wavelength'],
         radio.QE_rsr['response'])
plt.ylim(0, 1)

FigureCanvasNbAgg()

(0, 1)

In [20]:
plt.figure()
plt.plot(radio.QE_rsr['wavelength'],
         radio.QE_rsr['response'])
plt.plot(radio.waves,
         radio.QE_ipol, '--', color='red')

FigureCanvasNbAgg()

In [21]:
plt.figure()
plt.plot(radio.rsr['wavelength'],
         radio.rsr['response'])
plt.plot(radio.waves,
         radio.resp_ipol, '--', color='red')

FigureCanvasNbAgg()

In [22]:
from pytelescope.ccd import CCD

In [7]:
class Camera(object):
    def __init__(self, compression=5, fov=60,
                 n_bandpasses=4, **kwargs):

        self.compression = compression
        self.fov = fov * u.deg
        self.n_bandpasses = n_bandpasses
        self.ccd = CCD(**kwargs)

    def __getattr__(self, attr):
        return getattr(self.ccd, attr)

    @property
    def ifov(self):
        ifovx = (self.fov / self.ccd.x).to(u.rad)
        return ifovx
#         ifovy = (self.fov / self.ccd.y).to(u.rad)
#         return np.array([ifovx, ifovy])

    def calc_swath_from_alt(self, h):
        return 2*h*np.tan(self.fov.to(u.rad)/2)
    
    def calc_pixel_size_from_alt(self, h):
        return 2*h*np.tan(self.ifov/2)
        
    @property
    def ifov_mrad(self):
        return self.ifov.to(u.mrad)

    @property
    def img_compressed_size(self):
        return self.ccd.total_mbits / self.compression

    @property
    def img_set_size(self):
        return self.n_bandpasses * self.img_compressed_size

    def __repr__(self):
        s = self.ccd.__str__()
        s += "Compression: {}\n".format(self.compression)
        s += "Compressed per image: {} Mbits\n".format(
            self.img_compressed_size)
        s += "Bands: {}\n".format(self.n_bandpasses)
        s += "Set size compressed: {} Mbits\n".format(self.img_set_size)
        s += f"IFOV [mrad]: {self.ifov.to(u.mrad):g}\n"
        return s


class SolarIrradiance:
    def __init__(self, wave1=200 * u.nm, wave2=1200 * u.nm, dlambda=1 * u.nm):
        self.wave1 = wave1
        self.wave2 = wave2
        self.dlambda = dlambda
        sol = SolarIrradianceSpectrum(TOTAL_IRRADIANCE_SPECTRUM_2000ASTM)
        sol.interpolate(dlambda=dlambda.to(u.micron).value,
                        ival_wavelength=(wave1.to(u.micron).value,
                                         wave2.to(u.micron).value))
        self.waves = sol.ipol_wavelength * u.micron  # b/c pyspectral works in micron

In [117]:
cam = Camera(fov=8.5, n_bandpasses=5)

In [109]:
cam

2048 x 2048 pixels per CCD
N pixels: 4194304
Dynamic range: 15 bits
Total Mbits per CCD: 60.0
Compression: 5
Compressed per image: 12.0 Mbits
Bands: 5
Set size compressed: 60.0 Mbits
IFOV [mrad]: 0.0681769 mrad

In [104]:
altitudes = np.arange(40000,100001, 10000)

In [105]:
df = pd.DataFrame({'altitudes': altitudes * u.km})

In [118]:
cam.calc_swath_from_alt(83000*u.km)

<Quantity 12335.93089903 km>

In [119]:
cam.calc_pixel_size_from_alt(83000*u.km)

<Quantity 6.01235248 km>

In [82]:
df['swath_widths_10deg'] = [i.value for i in df.altitudes.map(cam.calc_swath_from_alt)]

In [83]:
df

,altitudes,swath_widths_8deg,swath_widths_10deg
0,40000.0,5594.144955,6999.093082
1,50000.0,6992.681194,8748.866353
2,60000.0,8391.217433,10498.639623
3,70000.0,9789.753672,12248.412894
4,80000.0,11188.289911,13998.186164
5,90000.0,12586.826150,15747.959435
6,100000.0,13985.362389,17497.732705


In [86]:
plt.figure()
df.set_index('altitudes').plot()
plt.ylabel('Swath widths [km]')
plt.title("Swath widths with 2 different camera FOVs")
plt.savefig("swath_widths.pdf")

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [92]:
df['pixel_ground_size_8deg'] = [i.value for i in df.altitudes.map(cam.calc_pixel_size_from_alt)]

In [71]:
%matplotlib ipympl

In [ ]:
df

In [94]:
plt.figure()
df.set_index('altitudes').plot()
plt.ylabel('Pixel ground sizes [km]')
plt.title("Pixel ground sizes with 2 different camera FOVs")
plt.savefig("pixel_ground_sizes.pdf")

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [72]:
df.set_index('altitudes', inplace=True)

In [73]:
plt.figure()
df.swath_widths.plot(label='FOV=8 deg')
plt.xlabel('Altitudes [km]')
plt.ylabel('Swath Widths [km]')
plt.legend()
plt.savefig('swath_widths_8_degree.png', dpi=200)

FigureCanvasNbAgg()

In [74]:
plt.figure()
df.pixel_ground_size.plot(label='FOV=8 deg')
plt.xlabel('Altitudes [km]')
plt.ylabel('Pixel Ground Sizes [km]')
plt.legend()
plt.savefig('pixel_sizes_8_degree.png')

FigureCanvasNbAgg()

In [55]:
df

,swath_widths,pixel_ground_size
altitudes,,
40000.0,6999.093082,3.408846
50000.0,8748.866353,4.261058
60000.0,10498.639623,5.113269
70000.0,12248.412894,5.965481
80000.0,13998.186164,6.817692
90000.0,15747.959435,7.669904
100000.0,17497.732705,8.522115


In [11]:
cam.calc_pixel_size_from_alt(100000*u.km)

<Quantity 8.52211549 km>

In [138]:
cam.ifov

<Quantity 8.52211549e-05 rad>

In [175]:
cam.ifov_mrad

<Quantity 0.05965481 mrad>

In [176]:
cam.ifov.to(u.microradian)

<Quantity 59.65480842 urad>

In [206]:
orb = orbiter.VenusOrbiter(100000)

Assuming kilometers as unit for input parameter.


In [207]:
orb.v_surf

<Quantity 99.87451523 m / s>

In [209]:
orb.ground_travel(1*u.min)

<Quantity 5992.4709136 m>

In [211]:
orb.T.to(u.hour)

<Quantity 105.75654403 h>

In [212]:
105/24

4.375

In [142]:
df = pd.read_csv("/Users/klay6683/Documents/VENUS/VDO/low_mid_lat_albedo.csv")

In [143]:
df2 = pd.read_csv("/Users/klay6683/Documents/VENUS/VDO/polar_region_albedo.csv")

In [144]:
df.head()

,Wavelength[nm],Albedo
0,251.607941,0.517896
1,253.567887,0.509862
2,255.673600,0.501351
3,258.402993,0.490016
4,261.302973,0.478793


In [145]:
df['polar'] = df2.Albedo

In [146]:
df.head()

,Wavelength[nm],Albedo,polar
0,251.607941,0.517896,0.451713
1,253.567887,0.509862,0.461598
2,255.673600,0.501351,0.433146
3,258.402993,0.490016,0.424981
4,261.302973,0.478793,0.401719


In [147]:
df.rename({'Albedo': 'Low-mid lats'}, axis=1, inplace=True)

In [148]:
df.head()

,Wavelength[nm],Low-mid lats,polar
0,251.607941,0.517896,0.451713
1,253.567887,0.509862,0.461598
2,255.673600,0.501351,0.433146
3,258.402993,0.490016,0.424981
4,261.302973,0.478793,0.401719


In [151]:
df.set_index('Wavelength[nm]').plot(title='Albedos from Lee, Titov, et al., 2015')
plt.savefig("albedos.pdf")

FigureCanvasNbAgg()

In [96]:
dr = 500 * u.kbit / u.s

In [97]:
dr

<Quantity 500. kbit / s>

In [101]:
(dr * 8 * u.h).to(u.Gbit)

<Quantity 14.4 Gbit>